# Parse the spreadsheet information

In [1]:
from pandas import read_excel

clade_mapping = read_excel("MAGs_toKBase_formodels.xlsx")
genera_to_track = ["Planktophila", "Methylopumilus", "Polynucleobacter", "Pirellula_B", "UBA3064", "UBA954"]


In [2]:
Planktophila, Methylopumilus, Polynucleobacter, Pirellula_B, UBA3064, UBA954 = [], [], [], [], [], [] 
other = []

for index, row in clade_mapping.iterrows():
    if not isinstance(row["classification"], str):  continue
    if "Planktophila" in row["classification"]:  Planktophila.append(row["user_genome"])
    elif "Methylopumilus" in row["classification"]:  Methylopumilus.append(row["user_genome"])
    elif "Polynucleobacter" in row["classification"]:  Polynucleobacter.append(row["user_genome"])
    elif "Pirellula_B" in row["classification"]:  Pirellula_B.append(row["user_genome"])
    elif "UBA3064" in row["classification"]:  UBA3064.append(row["user_genome"])
    elif "UBA954" in row["classification"]:  UBA954.append(row["user_genome"])
    else:  other.append(row["user_genome"])

In [2]:
clade_mapping

,user_genome,classification,dRep winner,Model
0,NASQAN2010_127_F_bin.3,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Y,Y
1,NASQAN2010_155_B_bin.36,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Y,Y
2,NASQAN2011_265_F_bin.15,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Y,Y
3,NASQAN2011_330_F_bin.10,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Y,Y
4,NASQAN2012_130_B_bin.14,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Y,Y
...,...,...,...,...
3822,NASQAN2012_115_A_bin.3,d__Bacteria;p__Acidobacteriota;c__Holophagae;o...,N,N
3823,tombigeeriver_2019_sw_WHONDRS-S19S_0020_B_bin.104,d__Bacteria;p__Acidobacteriota;c__Holophagae;o...,N,N
3824,NASQAN2011_262_A_bin.12,d__Bacteria;p__Acidobacteriota;c__Blastocatell...,N,N
3825,NaN,NaN,NaN,N


In [4]:
taxa = [taxon.split(";") for taxon in clade_mapping["classification"] if isinstance(taxon, str)]
for domain, phylum, Class, order, family, genus, species in taxa:
    if 

['d__Bacteria',
 'p__Proteobacteria',
 'c__Gammaproteobacteria',
 'o__Burkholderiales',
 'f__Methylophilaceae',
 'g__Methylopumilus',
 's__Methylopumilus universalis']

In [3]:
print(len(other))
print(list(map(len, [Planktophila, Methylopumilus, Polynucleobacter, Pirellula_B, UBA3064, UBA954])))

3406
[122, 81, 127, 6, 47, 36]


In [5]:
print("Planktophila =", Planktophila, "\n")
print("Methylopumilus =", Methylopumilus, "\n")
print("Polynucleobacter =", Polynucleobacter, "\n")
print("Pirellula_B =", Pirellula_B, "\n")
print("UBA3064 =", UBA3064, "\n")
print("UBA954 =", UBA954, "\n")

Planktophila = ['NASQAN2014_188_F_bin.12', 'NASQAN2016_164_F_bin.24', 'NASQAN2011_330_B_bin.17', 'NASQAN2014_193_B_bin.9', 'NASQAN2014_193_B_bin.83', 'NASQAN2015_208_B_bin.25', 'yakimariver_2019_sw_WHONDRS-S19S_0060_A_bin.25', 'NASQAN2014_190_A_bin.31', 'NASQAN2015_172_B_bin.57', 'NASQAN2015_174_B_bin.17', 'NASQAN2015_227_F_bin.3', 'pamunkeyriver_2019_sw_WHONDRS-S19S_0054_F_bin.9', 'NASQAN2011_057_B_bin.7', 'NASQAN2011_250_B_bin.9', 'NASQAN2011_250_F_bin.9', 'NASQAN2011_256_B_bin.24', 'NASQAN2011_265_F_bin.3', 'NASQAN2011_286_B_bin.20', 'NASQAN2011_286_F_bin.2', 'NASQAN2014_144_F_bin.23', 'NASQAN2014_188_A_bin.31', 'NASQAN2014_193_B_bin.55', 'NASQAN2015_047_F_bin.5', 'NASQAN2015_047_F_bin.9', 'NASQAN2015_052_B_bin.57', 'NASQAN2015_052_F_bin.15', 'NASQAN2015_052_F_bin.18', 'NASQAN2015_111_B_bin.69', 'NASQAN2015_150_F_bin.7', 'NASQAN2015_172_F_bin.7', 'NASQAN2015_174_F_bin.13', 'NASQAN2015_208_B_bin.21', 'NASQAN2015_208_F_bin.14', 'NASQAN2016_024_B_bin.5', 'NASQAN2016_164_A_bin.16', 'alt

# loading models

In [4]:
import os
os.environ["HOME"] = "/Users/afreiburger/Documents/cobrakbase/" # os.path.join('C:', 'Users', 'Andrew Freiburger','Documents','Argonne','cobrakbase')
import cobrakbase
# with open("D:/Users/Andrew Freiburger/Documents/Argonne/kbase_token.txt") as token_file:
with open("/Users/afreiburger/Documents/kbase_token.txt") as token_file:
    kbase_api = cobrakbase.KBaseAPI(token_file.readline())

modelseedpy 0.2.2
cobrakbase 0.3.1


In [5]:
models = kbase_api.list_objects(165689, object_type="KBaseFBA.FBAModel", include_metadata=False)
print(len(models))

3534


In [6]:
all_organisms = [ele for ls in [Planktophila, Methylopumilus, Polynucleobacter, Pirellula_B, UBA3064, UBA954] for ele in ls]

In [12]:
print(all_organisms)

['NASQAN2014_188_F_bin.12', 'NASQAN2016_164_F_bin.24', 'NASQAN2011_330_B_bin.17', 'NASQAN2014_193_B_bin.9', 'NASQAN2014_193_B_bin.83', 'NASQAN2015_208_B_bin.25', 'yakimariver_2019_sw_WHONDRS-S19S_0060_A_bin.25', 'NASQAN2014_190_A_bin.31', 'NASQAN2015_172_B_bin.57', 'NASQAN2015_174_B_bin.17', 'NASQAN2015_227_F_bin.3', 'pamunkeyriver_2019_sw_WHONDRS-S19S_0054_F_bin.9', 'NASQAN2011_057_B_bin.7', 'NASQAN2011_250_B_bin.9', 'NASQAN2011_250_F_bin.9', 'NASQAN2011_256_B_bin.24', 'NASQAN2011_265_F_bin.3', 'NASQAN2011_286_B_bin.20', 'NASQAN2011_286_F_bin.2', 'NASQAN2014_144_F_bin.23', 'NASQAN2014_188_A_bin.31', 'NASQAN2014_193_B_bin.55', 'NASQAN2015_047_F_bin.5', 'NASQAN2015_047_F_bin.9', 'NASQAN2015_052_B_bin.57', 'NASQAN2015_052_F_bin.15', 'NASQAN2015_052_F_bin.18', 'NASQAN2015_111_B_bin.69', 'NASQAN2015_150_F_bin.7', 'NASQAN2015_172_F_bin.7', 'NASQAN2015_174_F_bin.13', 'NASQAN2015_208_B_bin.21', 'NASQAN2015_208_F_bin.14', 'NASQAN2016_024_B_bin.5', 'NASQAN2016_164_A_bin.16', 'altamaha_2019_sw_W

In [7]:
name_to_id = {obj[1].replace(".RAST.mdl", ""): f"165689/{obj[0]}/{obj[4]}" for obj in models}
name_to_id

{'NASQAN2014_193_B_bin.7': '165689/5/11',
 'hopbrook_2019_sw_WHONDRS-S19S_0012_B_bin.27': '165689/7/11',
 'NASQAN2014_188_B_bin.31': '165689/31/1',
 'pamunkeyriver_2019_sw_WHONDRS-S19S_0054_F_bin.15': '165689/33/1',
 'morescreek_2019_sw_WHONDRS-S19S_0085_B_bin.26': '165689/35/1',
 'NASQAN2016_083_B_bin.83': '165689/36/1',
 'redbuttecreek_2019_sw_WHONDRS-S19S_0029_B_bin.5': '165689/38/1',
 'columbiariver1_2019_sw_WHONDRS-S19S_0075_A_bin.47': '165689/40/1',
 'NASQAN2016_164_B_bin.38': '165689/41/1',
 'dobsoncreek_2019_sw_WHONDRS-S19S_0087_A_bin.18': '165689/45/1',
 'watershed3_2019_sw_WHONDRS-S19S_0084_A_bin.2': '165689/46/1',
 'nebranchanacostia_2019_sw_WHONDRS-S19S_0081_A_bin.24': '165689/49/1',
 'blueriver_2019_sw_WHONDRS-S19S_0024_A_bin.40': '165689/51/1',
 'eastriver_2019_sw_WHONDRS-S19S_0100_B_bin.14': '165689/53/1',
 'riogrande_2019_sw_WHONDRS-S19S_0066_B_bin.158': '165689/55/1',
 'greatmiami_2019_sw_WHONDRS-S19S_0099_B_bin.93': '165689/57/1',
 'lowercolorado_2019_sw_WHONDRS-S19S_

In [8]:
Planktophila = {k:v for k,v in name_to_id.items() if k in Planktophila}
Methylopumilus = {k:v for k,v in name_to_id.items() if k in Methylopumilus}
Polynucleobacter = {k:v for k,v in name_to_id.items() if k in Polynucleobacter}
Pirellula_B = {k:v for k,v in name_to_id.items() if k in Pirellula_B}
UBA3064 = {k:v for k,v in name_to_id.items() if k in UBA3064}
UBA954 = {k:v for k,v in name_to_id.items() if k in UBA954}

In [9]:
constructed_models = []
for model in models:
    genome_name = model[1].replace(".RAST.mdl", "")
    if genome_name in all_organisms:
        constructed_models.append(genome_name)

missing_models = set(all_organisms)-set(constructed_models)
print("missing models", len(missing_models))

missing models 4


In [10]:
print([all([organism in constructed_models for organism in ls]) for ls in [Planktophila, Methylopumilus, Polynucleobacter, Pirellula_B, UBA3064, UBA954]])

[True, True, True, True, True, True]


In [106]:
models[0]

[5,
 'NASQAN2014_193_B_bin.7.RAST.mdl',
 'KBaseFBA.FBAModel-15.0',
 '2023-12-14T01:01:15+0000',
 11,
 'jplfaria',
 165689,
 'jplfaria:narrative_1702331977402',
 '962568a3950afb93ca2d84ca8115fbfc',
 1334936,
 None]

In [95]:
Planktophila

{'NASQAN2014_188_B_bin.31': '165689/31/1',
 'NASQAN2011_286_F_bin.2': '165689/100/1',
 'lowercolorado_2019_sw_WHONDRS-S19S_0063_A_bin.57': '165689/246/1',
 'pamunkeyriver_2019_sw_WHONDRS-S19S_0054_F_bin.9': '165689/282/1',
 'pamunkeyriver_2019_sw_WHONDRS-S19S_0054_B_bin.101': '165689/322/1',
 'lowercolorado_2019_sw_WHONDRS-S19S_0063_A_bin.80': '165689/430/1',
 'kuparukriver_2019_sw_WHONDRS-S19S_0047_F_bin.5': '165689/502/1',
 'NASQAN2015_111_B_bin.69': '165689/559/1',
 'deadrun_2019_sw_WHONDRS-S19S_0006_F_bin.23': '165689/566/1',
 'oksrukuyikcreek_2019_sw_WHONDRS-S19S_0050_B_bin.27': '165689/577/1',
 'icacos_2019_sw_WHONDRS-S19S_0094_F_bin.22': '165689/698/1',
 'NASQAN2015_227_F_bin.3': '165689/719/1',
 'NASQAN2014_188_F_bin.12': '165689/774/1',
 'oksrukuyikcreek_2019_sw_WHONDRS-S19S_0050_B_bin.36': '165689/799/1',
 'lowercolorado_2019_sw_WHONDRS-S19S_0063_F_bin.26': '165689/957/1',
 'NASQAN2011_250_B_bin.9': '165689/977/1',
 'NASQAN2011_060_B_bin.40': '165689/1027/1',
 'pringlecreek_2

In [116]:
clades_models = {"Planktophila": Planktophila, "Methylopumilus": Methylopumilus, "Polynucleobacter": Polynucleobacter, 
                 "Pirellula_B": Pirellula_B, "UBA3064": UBA3064, "UBA954": UBA954}

# Construct the clade models

In [12]:
%run ../../ModelSEEDpy/modelseedpy/core/msprobability.py

clade_model = MSProbability.megaModel({"Pirellula_B": list(Pirellula_B.values())}, kbase_api) #, "Pirellula_B.json")

165689/6616/2	index 3												
 Pirellula_B
missing reactions:  {'numMembers'}


In [150]:
from cobra.io import to_json
from json import dump

model_json = to_json(clade_model[0])
with open("test_json.json", "w") as jsonOut:   dump(model_json, jsonOut, indent = 3)

test_json_model = load_json_model("test_json.json")

TypeError: string indices must be integers

In [13]:
from cobra.io import read_sbml_model
sample_model = read_sbml_model("reaction_counts/Pirellula_B.xml")

In [132]:
from optlang import Objective
sample_model.objective = Objective(sample_model.reactions.bio1.flux_expression, direction="max")

In [2]:
sample_model.slim_optimize()

57.59369610481184

In [16]:
print(dir(sample_model.reactions[0]))

['__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__isub__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_annotation', '_associate_gene', '_check_bounds', '_dissociate_gene', '_genes', '_gpr', '_id', '_lower_bound', '_metabolites', '_model', '_repr_html_', '_set_id_with_model', '_update_awareness', '_upper_bound', 'add_metabolites', 'annotation', 'boundary', 'bounds', 'build_reaction_from_string', 'build_reaction_string', 'check_mass_balance', 'compartments', 'copy', 'delete', 'flux', 'flux_expression', 'forward_variable', 'functional', 'gene_name_reaction_rule', 'gene_reaction_rule', 'genes', 'get_coefficient'

In [145]:
for rxn in sample_model.reactions:
    rxn.probability = 0.5
    # print(rxn.probability)

print(dir(sample_model.reactions[0]))

from cobra.io import save_json_model
save_json_model(sample_model, "probability_attribute_test.json")

['__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__isub__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_annotation', '_associate_gene', '_check_bounds', '_dissociate_gene', '_genes', '_gpr', '_id', '_lower_bound', '_metabolites', '_model', '_repr_html_', '_set_id_with_model', '_update_awareness', '_upper_bound', 'add_metabolites', 'annotation', 'boundary', 'bounds', 'build_reaction_from_string', 'build_reaction_string', 'check_mass_balance', 'compartments', 'copy', 'delete', 'flux', 'flux_expression', 'forward_variable', 'functional', 'gene_name_reaction_rule', 'gene_reaction_rule', 'genes', 'get_coefficient'

In [146]:
from cobra.io import load_json_model
probability_attribute_test = load_json_model("probability_attribute_test.json")
print(dir(probability_attribute_test.reactions[0]))

['__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__isub__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_annotation', '_associate_gene', '_check_bounds', '_dissociate_gene', '_genes', '_gpr', '_id', '_lower_bound', '_metabolites', '_model', '_repr_html_', '_set_id_with_model', '_update_awareness', '_upper_bound', 'add_metabolites', 'annotation', 'boundary', 'bounds', 'build_reaction_from_string', 'build_reaction_string', 'check_mass_balance', 'compartments', 'copy', 'delete', 'flux', 'flux_expression', 'forward_variable', 'functional', 'gene_name_reaction_rule', 'gene_reaction_rule', 'genes', 'get_coefficient'

In [14]:
# from cobra.io import read_sbml_model
from cobrakbase.core.kbasefba.fbamodel_from_cobra import CobraModelConverter
cobra_prob_test = CobraModelConverter(sample_model).build()
print(cobra_prob_test.slim_optimize())

0.0


In [18]:
print(cobra_prob_test.optimize())

<Solution 0.000 at 0x13a74be20>


In [27]:
cobra_prob_test.reactions.rxn00006_c0.probability

0.25

In [4]:
print(dir(cobra_prob_test.reactions[0]))
print(cobra_prob_test.reactions[0].notes)

['__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__isub__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_annotation', '_associate_gene', '_check_bounds', '_dissociate_gene', '_genes', '_get_dna', '_get_rna', '_gpr', '_id', '_lower_bound', '_metabolites', '_model', '_repr_html_', '_set_dna', '_set_id_with_model', '_set_rna', '_to_json', '_update_awareness', '_upper_bound', 'add_metabolites', 'annotation', 'boundary', 'bounds', 'build_reaction_from_string', 'build_reaction_string', 'cell_wall', 'check_mass_balance', 'cofactor', 'compartments', 'copy', 'delete', 'dna', 'energy', 'flux', 'flux_expression', 'forwar

In [147]:
print(type(probability_attribute_test))

<class 'cobra.core.model.Model'>


In [142]:
%run ../../ModelSEEDpy/modelseedpy/core/msprobability.py

MSProbability.apply_threshold(sample_model, 0.5)

AttributeError: 'Reaction' object has no attribute 'probability'